In [1]:
!pip install mtcnn

     |████████████████████████████████| 2.3 MB 5.3 MB/s 


In [2]:
#import math
import os
import imutils
import numpy as np
import pandas as pd
import cv2 as cv2
import glob
#import csv
#from sklearn import datasets
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import LinearSVC
#from sklearn.model_selection import KFold
#from sklearn.metrics import confusion_matrix,classification_report
#from sklearn.metrics import roc_curve, auc
#from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
#from itertools import cycle
#from scipy import interp
#from sklearn.multiclass import OneVsRestClassifier
#import random
#from scipy import ndarray
import skimage as sk
from skimage import transform
#from skimage import util
#from sklearn.svm import SVC
#from skimage import io
import copy
from mtcnn.mtcnn import MTCNN 
from skimage import feature

In [3]:
from google.colab import drive
drive.mount("/content/drive/",force_remount=True)

Mounted at /content/drive/


## Data collection

In [4]:
def _process_row( row ):
    """
    Process a single dataframe row, returns the argmax label
    :param row:
    :return:
    """
    return np.argmax( row )

In [5]:
main_path = '/content/drive/MyDrive/FER/Data/Images'
train_path = os.path.join( main_path, 'FER2013Train' )
test_path = os.path.join( main_path, 'FER2013Test' )
valid_path = os.path.join( main_path, 'FER2013Valid' )

#initialize dataframe FER+ with actuall images in folder (some might be missing or duplicates)
def get_DataFrame( path_name ):
    label_file = os.path.join(path_name, 'label.csv')
    data_df = pd.read_csv(label_file, header=None)
    data_df.columns = ["img_name", "dims", "0", "1", "2", "3", "4", "5", "6", "7","Unknown", "NF"]
    data_df['actual_label'] = data_df[['0', '1', '2', '3', '4', '5', '6', '7']].apply(lambda x: _process_row(x), axis=1)

    # get all ilocs with actual label 0
    data_df = data_df.sort_values(by=['img_name'])
    image_file_names = data_df['img_name'].values
    d_files = [ filename.split('/')[-1] for filename in glob.glob(path_name+'/*.*') ]
    data_df['exist_file'] = [ 1 if ii in d_files else np.nan for ii in image_file_names ]
    return data_df.dropna()

#reads image in cv2 format, transforms to grayscale since many operations needs this
def get_Image( path ):
    return cv2.imread( path, cv2.IMREAD_GRAYSCALE )

#converts gray image to string
def _grayimg2string( img ):
    return str(img.flatten().tolist()).replace(']','').replace('[','')

#converts string to gray image
def _string2grayimage( img, size=(48,48) ):
    return np.array(img.split(', ')).astype(np.uint8).reshape( size )


#creates dataframes per emotion loading images in gray scale
def get_emotionsDataFrames( path_name, data_df ):
    for ii in data_df['actual_label'].unique():
        dd = data_df[data_df['actual_label']==ii][['img_name','actual_label']]
        dd['img_str'] = dd['img_name'].apply(lambda x: _grayimg2string( get_Image( path_name+'/'+x ) ) )
        dd.to_csv(path_name+'/'+f'labels_{ii}.csv', index=False)
    return        

In [7]:
get_emotionsDataFrames( train_path, get_DataFrame( train_path ) )
get_emotionsDataFrames( test_path, get_DataFrame( test_path ) )
get_emotionsDataFrames( valid_path, get_DataFrame( valid_path ) )

## Data cleaning

In [8]:
#For plotting image
def show( img ):
    plt.imshow( img, cmap='gray' )
    return

#Cleaning noise
def Denoising( img, h=10, templateWindowSize=7, searchWindowSize=21 ):
    return cv2.fastNlMeansDenoising(img,None,h,templateWindowSize,searchWindowSize)

In [9]:
#detect faces with mtcnn
# we use this library after reading https://towardsdatascience.com/face-detection-models-which-to-use-and-why-d263e82c302c 
def ReturnLocalizedFace( img ):
    detector = MTCNN()
    faces = detector.detect_faces( cv2.cvtColor( img, cv2.COLOR_GRAY2RGB ) )
    return faces

#draw faces result from mtcnn (https://towardsdatascience.com/face-detection-using-mtcnn-a-guide-for-face-extraction-with-a-focus-on-speed-c6d59f82d49)
def DrawFacesMTCNN( img, faces ):
    plt.imshow( img, cmap='gray' )
    ax = plt.gca()
    for face in faces:
        x, y, w, h = face['box']
        rect = plt.Rectangle( (x,y), w, h , fill=False, color='green', lw=1 )
        ax.add_patch( rect )
        for key, value in face['keypoints'].items():
            dot = plt.Circle( value, radius=3, color='red' )
            ax.add_patch( dot )
    plt.show()
    return

#detect faces and uses eye-eye position to rotates and match a given "standard" 
def RotatetoFrontalFace( img, faces ):
    #target positions from image FER2013Test/fer0032232.png
    le_t = np.array([13,18],dtype=float)
    re_t = np.array([35,18],dtype=float)
    no_t = np.array([24,30],dtype=float)
    ml_t = np.array([17,41],dtype=float)
    mr_t = np.array([31,41],dtype=float)
    eye_separation = re_t[0]-le_t[0]
    
    #faces = ReturnLocalizedFace( img )
    if len(faces)!=1:
        return None
    
    le=faces[0].get('keypoints').get('left_eye')
    if not le:
        return None
    le=np.array(le,dtype=float)
    re=faces[0].get('keypoints').get('right_eye')
    if not re:
        return None    
    re=np.array(re,dtype=float)
    
    lr = re - le
    lr_n = np.linalg.norm(lr)
    c_alph = lr[0]/lr_n
    sgn = np.sign(lr[1])
    s_alph = sgn*np.sqrt(1-c_alph**2)
    scale = eye_separation/lr_n
    
    rot_mat = np.array([[  scale*c_alph,  scale*s_alph,  -scale*c_alph*le[0]+le_t[0]-scale*s_alph*le[1] ],
                      [ -scale*s_alph,  scale*c_alph,  -scale*c_alph*le[1]+le_t[1]+scale*s_alph*le[0] ]])
    
    new_img = cv2.warpAffine( img, rot_mat)
    #return crop image in size of "standard"
    return new_img[0:48, 0:48]

## Feature extraction
Here we assume that:
1. A an image **img** was read with  **get_Image** (or loaded from new dataframes, ie 'labels_0.csv')
2. Then **img = Denoising( img )**
3. **faces = ReturnLocalizedFace( img )** 
4. **img = RotatetoFrontalFace( img, faces )** 
before extracting features.

In [10]:
#Extracts HOG
def ExtractHOG( img, win_size=(64,128) ):    
    img = cv2.resize( img, win_size )
    #hog
    d = cv2.HOGDescriptor()
    hog = d.compute( img )
    return hog
    
#Extract Gabor 
# based on https://cvtuts.wordpress.com/2014/04/27/gabor-filters-a-practical-overview/
# TODO (change to frequency domain as in https://www.ini.rub.de/upload/file/1470692845_33efbf50567f9d637771/LadEtAl1993.pdf
#        and file:///tmp/mozilla_gabriel0/ICCV2005-ZhangShan-LGBP.pdf)
def BuildGaborFilters( ksize=11, sigma=3, lambd=9, gamma=0.5, psi=0. ):
    filters0=[]
    filters1=[]
    for theta in np.arange( 0, np.pi, np.pi/8 ):
        kern0 = cv2.getGaborKernel( (ksize,ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        kern0 /= 1.5*kern0.sum()
        kern1 = cv2.getGaborKernel( (ksize,ksize), sigma/np.sqrt(2), theta, lambd/np.sqrt(2), gamma, psi, ktype=cv2.CV_32F)
        kern1 /= 1.5*kern0.sum()
        filters0.append(kern0)
        filters1.append(kern1)
    
    return filters0, filters1

filters = BuildGaborFilters()

def ProcessGabor( img, filters ):
    accum0 = np.zeros_like(img)
    accum1 = np.zeros_like(img)
    for kern0 in filters[0]:
        fimg = cv2.filter2D( img, cv2.CV_8UC3, kern0 )
        np.maximum( accum0, fimg, accum0 )
    for kern1 in filters[1]:
        fimg = cv2.filter2D( img, cv2.CV_8UC3, kern1 )
        np.maximum( accum1, fimg, accum1 )    
    
    return accum0, accum1
    
#Extract LBP Features 
# divides in 7X7 squares as in https://www.pyimagesearch.com/2021/05/03/face-recognition-with-local-binary-patterns-lbps-and-opencv/
# class based in https://www.pyimagesearch.com/2015/12/07/local-binary-patterns-with-python-opencv/

# ALTERNATIVE (use overlaping planes as in Figure 1 of http://www.scholarpedia.org/article/Local_Binary_Patterns
#      ref https://d1wqtxts1xzle7.cloudfront.net/39765451/Dynamic_Texture_Recognition_Using_Local_20151106-14680-19lgjsz-with-cover-page-v2.pdf?Expires=1636851787&Signature=F00Aon5sbz1RCybxye5lrG~AV~Xw6DTPn79CRtsNGJMNSKWb8~JkkIRA9~9FtNnO8AgumGrhfoGq1VJHlUbGCkQb9incCas-7kZlrtFLJCsu3R~d114P2vkphi-TkqKdwvgpre9s6WMynHBgseCrMT9vEvP38uz3a4BC16qrUaP~BENzgD-HbSW88VlGaGfrToapU-tP-dDGR6k5j2ctjz-y3anfa2qronHqgUa4p7HgJUdnKhYQzm3fSj01A~r5y0EzoL3OqTNNRl8Uxezu-9V~acDYT4AQvl3KiAbQRNxmnRfYbhOFwJo3lMppdUuyYmQi7MPHZGvnrQMT8Z07YA__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA 
#      )
class LocalBinaryPatterns:
    def __init__( self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius
        
    def describe(self, image, eps=1e-7):    
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        # return the histogram of Local Binary Patterns
        return hist

LBPdesc = LocalBinaryPatterns(8, 1)

def ExtractLBPHist( img, LBPdesc ):
    LBP = []
    for ii in range(0,7):
        for jj in range(0,7):
            hist = LBPdesc.describe( img[ ii*7:(ii+1)*7, jj*7:(jj+1)*7 ] )
            LBP.append(hist)
    return LBP